In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt

# 设置中文字体支持
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]

def load_yolo_annotations(annotation_path, image_width, image_height):
    """
    加载YOLO格式的标注文件
    返回值: 标注框列表，每个框为(x1, y1, x2, y2)
    """
    annotations = []
    if not os.path.exists(annotation_path):
        return annotations

    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue  # 跳过格式不正确的行

        # YOLO格式: class_id center_x(归一化) center_y(归一化) width(归一化) height(归一化)
        class_id, cx, cy, w, h = map(float, parts)

        # 转换为像素坐标
        cx_pixel = cx * image_width
        cy_pixel = cy * image_height
        w_pixel = w * image_width
        h_pixel = h * image_height

        # 计算左上角和右下角坐标
        x1 = int(cx_pixel - w_pixel / 2)
        y1 = int(cy_pixel - h_pixel / 2)
        x2 = int(cx_pixel + w_pixel / 2)
        y2 = int(cy_pixel + h_pixel / 2)

        annotations.append((x1, y1, x2, y2))

    return annotations

def draw_annotations(image, annotations, color=(0, 255, 0)):
    """绘制人工标注框（绿色）"""
    img = image.copy()
    for (x1, y1, x2, y2) in annotations:
        # 绘制矩形框
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    return img

def draw_detections(image, results, conf_threshold=0.5, color=(0, 0, 255)):
    """绘制检测结果框（红色），不显示标签"""
    img = image.copy()
    for result in results:
        for box in result.boxes:
            if box.conf[0] >= conf_threshold:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # 绘制矩形框，不添加标签
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    return img

def save_visualizations(model, img_dir, label_dir, save_dir, conf_threshold=0.5):
    """保存人工标注和检测结果的可视化图像"""
    # 创建保存目录
    save_dir.mkdir(parents=True, exist_ok=True)

    # 获取所有图像文件
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    img_files = [f for f in os.listdir(img_dir) if Path(f).suffix.lower() in img_extensions]
    total = len(img_files)

    for i, img_file in enumerate(img_files):
        # 进度提示
        if i % 10 == 0:
            print(f"处理中: {i}/{total} 图像")

        # 图像路径
        img_path = os.path.join(img_dir, img_file)
        # 对应的标注文件路径
        label_file = Path(img_file).stem + '.txt'
        label_path = os.path.join(label_dir, label_file)

        # 读取图像
        image = cv2.imread(img_path)
        if image is None:
            print(f"警告: 无法读取图像 {img_path}")
            continue

        # 获取图像尺寸
        image_height, image_width = image.shape[:2]
        # 转换颜色通道（BGR -> RGB）
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 加载并绘制人工标注
        annotations = load_yolo_annotations(label_path, image_width, image_height)
        ann_img = draw_annotations(image_rgb, annotations)

        # 保存人工标注可视化结果
        ann_save_path = save_dir / f"{Path(img_file).stem}_annotation.jpg"
        plt.imsave(ann_save_path, ann_img)

        # 模型检测
        results = model(img_path, imgsz=1024, conf=conf_threshold, verbose=False)

        # 绘制并保存检测结果
        det_img = draw_detections(image_rgb, results, conf_threshold)
        det_save_path = save_dir / f"{Path(img_file).stem}_detection.jpg"
        plt.imsave(det_save_path, det_img)

    print(f"所有图像处理完成，结果保存在 {save_dir}")

if __name__ == "__main__":
    # 1. 加载模型
    model = YOLO(r"D:\Models\YOLOS4A-l.pt")

    # 2. 设置路径（根据你的实际路径设置）
    img_dir = r"D:\Datas\VisDrone\VisDrone2019-DET-val\images"  # 图片文件夹
    label_dir = r"D:\Datas\VisDrone\VisDrone2019-DET-val\labels"  # 标注文件夹
    save_dir = Path("/vis_demo")  # 结果保存目录

    # 3. 处理并保存可视化结果，置信度设为0.5
    save_visualizations(model, img_dir, label_dir, save_dir, conf_threshold=0.5)


处理中: 0/548 图像
处理中: 10/548 图像
处理中: 20/548 图像
处理中: 30/548 图像
处理中: 40/548 图像
处理中: 50/548 图像
处理中: 60/548 图像
处理中: 70/548 图像
处理中: 80/548 图像
处理中: 90/548 图像
处理中: 100/548 图像
处理中: 110/548 图像
处理中: 120/548 图像
处理中: 130/548 图像
处理中: 140/548 图像
处理中: 150/548 图像
